In [0]:
import numpy as np
import pandas as pd

In [0]:
# Training set
df = pd.read_excel('trainset.xlsx')

In [0]:
df = df.drop(df.columns[1:3], axis = 1)
df = df.drop(df.columns[2], axis = 1)

In [0]:
train_y1 = []
for i in range(1,50):
    a = df[df[df.columns[2]] == i]
    a = a.drop(a.columns[2], axis = 1)
    a = a.set_index(a.columns[0])
    a = a[['CHEMICAL', 'ACID', 'AMMONIA/URINOUS', 'BAKERY', 'BURNT', 'COLD',
       'DECAYED', 'FISH', 'FLOWER', 'FRUIT', 'GARLIC', 'GRASS',
       'INTENSITY/STRENGTH', 'MUSKY', 'SOUR', 'SPICES', 'SWEATY', 'SWEET',
       'VALENCE/PLEASANTNESS', 'WARM', 'WOOD', 'Intensity']]
    train_y1.append(a)

In [0]:
train_y1[0].shape

(716, 22)

In [0]:
# Leaderboard set
df2 = pd.read_excel('trainset2.xlsx')
d = pd.read_excel('lb_dil.xlsx').set_index('oID')

In [0]:
train_y2 = []
for i in range(1,50):
    a = df2[df2.individual == i]
    a = a.drop(columns='individual')
    b = a.pivot(index = '#oID', columns='descriptor')
    b.columns = b.columns.droplevel(0)
    b = b.join(d)
    b = b[['CHEMICAL', 'ACID', 'AMMONIA/URINOUS', 'BAKERY', 'BURNT', 'COLD',
       'DECAYED', 'FISH', 'FLOWER', 'FRUIT', 'GARLIC', 'GRASS',
       'INTENSITY/STRENGTH', 'MUSKY', 'SOUR', 'SPICES', 'SWEATY', 'SWEET',
       'VALENCE/PLEASANTNESS', 'WARM', 'WOOD', 'Intensity']]
    train_y2.append(b)

In [0]:
train_y = []
for i in range(49):
    a = pd.concat([train_y1[0], train_y2[0]], sort = False)
    a = a.sort_index()
    train_y.append(a)

In [0]:
molecular = pd.read_excel('molecular.xlsx',index_col='CID')
train_X = molecular.loc[train_y[0].index.values]

In [0]:
molecular.shape

(476, 4869)

In [0]:
del train_y1
del train_y2

In [0]:
df3 = pd.read_excel('testset1.xlsx')
d = pd.read_excel('testset_d.xlsx').set_index('oID')
test_y = []
for i in range(1,50):
    a = df3[df3.individual == i]
    a = a.drop(columns='individual')
    b = a.pivot(index = '#oID', columns='descriptor')
    b.columns = b.columns.droplevel(0)
    b = b.join(d)
    b = b[['CHEMICAL', 'ACID', 'AMMONIA/URINOUS', 'BAKERY', 'BURNT', 'COLD',
       'DECAYED', 'FISH', 'FLOWER', 'FRUIT', 'GARLIC', 'GRASS',
       'INTENSITY/STRENGTH', 'MUSKY', 'SOUR', 'SPICES', 'SWEATY', 'SWEET',
       'VALENCE/PLEASANTNESS', 'WARM', 'WOOD', 'Intensity']]
    test_y.append(b)

In [0]:
test_X = molecular.loc[test_y[0].index.values]

In [0]:
test_y[0].shape

(69, 22)

In [0]:
train_y[0].shape

(785, 22)

In [0]:
train_X.shape

(785, 4869)

In [0]:
test_X.shape

(69, 4869)

In [0]:
test_s2 = test_y[0]
for i in range(1,49):
    test_s2 = pd.concat([test_s2, test_y[i]], sort = False)

In [0]:
test_s2 = test_s2.drop(columns=test_s2.columns[-1])
test_s2_mean = test_s2.groupby(test_s2.index).mean().values
test_s2_std = test_s2.groupby(test_s2.index).std().values

# Variable declare
train_X : train x

train_y: length:49, each element is trainset for one people

test_X : test x

test_y: length:49, each element is trainset for one people

# PCA

In [0]:
# Replace NAN to 0 and convert dataframe to Numpy array
train_X = train_X.fillna(0).values
for ind, df in enumerate(train_y):
    train_y[ind] = df.fillna(0).values
test_X = test_X.fillna(0).values
for ind, df in enumerate(test_y):
    test_y[ind] = df.fillna(0).values

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [0]:
# Normalization
scaler = StandardScaler()
train_X= scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [0]:
# PCA
pca = PCA(n_components=50)
train_X = pca.fit_transform(train_X)
test_X = pca.transform(test_X)

# Analysis

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

import xgboost as xgb

In [0]:
# Generate label for test set
label_1 = np.array(test_y)
label_1 = label_1[:, :, :-1]

label_2 = np.zeros((2, 69, 21))
label_2[0, :, :] = np.mean(label_1, axis=0)
label_2[1, :, :] = np.std(label_1, axis=0)

In [0]:
# Linear Regression
regr_lr = []
pred_lr = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_lr.append(LinearRegression())
    regr_lr[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_lr.append(regr_lr[-1].predict(test_X_cur))
pred_lr = np.array(pred_lr)
# pred_lr: list of 49 individuals, each is 69 x 21 np.array

In [0]:
pred_lr_s2_mean = np.mean(pred_lr, axis= 0)
pred_lr_s2_std = np.std(pred_lr, axis= 0)

In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_lr[i]))
print(np.mean(mse))

443.50922331491364


In [0]:
mse_s2_mean = mean_squared_error(pred_lr_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_lr_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

125.11022422674571
347.69194263063827


In [0]:
# RF
regr_rf = []
pred_rf = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_rf.append(RandomForestRegressor(n_estimators = 10,min_samples_split = 3, min_samples_leaf = 13))
    regr_rf[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_rf.append(regr_rf[-1].predict(test_X_cur))
pred_rf = np.array(pred_rf)

In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_rf[i]))
print(np.mean(mse))

433.50297481549154


In [0]:
pred_rf_s2_mean = np.mean(pred_rf, axis= 0)
pred_rf_s2_std = np.std(pred_rf, axis= 0)
mse_s2_mean = mean_squared_error(pred_rf_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_rf_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

106.81015447518796
279.53088655897983


In [0]:
# BayesianRidge
regr_br = []
pred_br = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_br.append(MultiOutputRegressor(BayesianRidge()))
    regr_br[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_br.append(regr_br[-1].predict(test_X_cur))
pred_br = np.array(pred_br)

In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_br[i]))
print(np.mean(mse))

400.6913771602526


In [0]:
pred_br_s2_mean = np.mean(pred_br, axis= 0)
pred_br_s2_std = np.std(pred_br, axis= 0)
mse_s2_mean = mean_squared_error(pred_br_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_br_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

81.47098456370092
347.69194263063827


In [0]:
# Lasso
regr_la = []
pred_la = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_la.append(MultiOutputRegressor(Lasso()))
    regr_la[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_la.append(regr_la[-1].predict(test_X_cur))
pred_la = np.array(pred_la)

In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_la[i]))
print(np.mean(mse))

431.17595198800853


In [0]:
pred_la_s2_mean = np.mean(pred_la, axis= 0)
pred_la_s2_std = np.std(pred_la, axis= 0)
mse_s2_mean = mean_squared_error(pred_la_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_la_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

112.2751510521671
347.69194263063827


In [0]:
# Ridge
regr_ri = []
pred_ri = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_ri.append(MultiOutputRegressor(Ridge()))
    regr_ri[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_ri.append(regr_ri[-1].predict(test_X_cur))
pred_ri = np.array(pred_ri)


In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_ri[i]))
print(np.mean(mse))

443.4650463214859


In [0]:
pred_ri_s2_mean = np.mean(pred_ri, axis= 0)
pred_ri_s2_std = np.std(pred_ri, axis= 0)
mse_s2_mean = mean_squared_error(pred_ri_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_ri_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

125.06615997722699
347.69194263063827


In [0]:
# xgb
regr_xg = []
pred_xg = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_xg.append(MultiOutputRegressor(xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 7, alpha = 10, n_estimators = 15)))
    regr_xg[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_xg.append(regr_xg[-1].predict(test_X_cur))
pred_xg = np.array(pred_xg)

In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_xg[i]))
print(np.mean(mse))

423.2290317420809


In [0]:
pred_xg_s2_mean = np.mean(pred_xg, axis= 0)
pred_xg_s2_std = np.std(pred_xg, axis= 0)
mse_s2_mean = mean_squared_error(pred_xg_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_xg_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

108.45508551570444
347.69184929584276


In [0]:
# SVR
regr_sv = []
pred_sv = []
for i in range(49):
    train_X_cur = np.concatenate((train_X, train_y[i][:, -1].reshape(-1, 1)), axis=1)
    train_X_cur[:, -1] = (train_X_cur[:, -1] - np.mean(train_X_cur[:, -1])) / np.std(train_X_cur[:, -1])
    test_X_cur = np.concatenate((test_X, test_y[i][:, -1].reshape(-1, 1)), axis=1)
    test_X_cur[:, -1] = (test_X_cur[:, -1] - np.mean(test_X_cur[:, -1])) / np.std(test_X_cur[:, -1])
    regr_sv.append(MultiOutputRegressor(SVR()))
    regr_sv[-1].fit(train_X_cur, train_y[i][:, :-1])
    pred_sv.append(regr_sv[-1].predict(test_X_cur))
pred_sv = np.array(pred_la)

In [0]:
mse = []
for i in range(49):
    mse.append(mean_squared_error(label_1[i], pred_sv[i]))
print(np.mean(mse))

431.17595198800853


In [0]:
pred_sv_s2_mean = np.mean(pred_sv, axis= 0)
pred_sv_s2_std = np.std(pred_sv, axis= 0)
mse_s2_mean = mean_squared_error(pred_sv_s2_mean,test_s2_mean)
mse_s2_std = mean_squared_error(pred_sv_s2_std, test_s2_std)
print(mse_s2_mean)
print(mse_s2_std)

112.2751510521671
347.69194263063827
